In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

workspace = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\near_streams_test\\"
arcpy.env.workspace = workspace


In [2]:
#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp"

#set source for streams data
streams = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_50c_streams_edited.shp"

#set source for subwatersheds
all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\50sws_10groups\all_10_groups.shp"

In [ ]:
#get distance from residential buildings to streams
#project buildings and streams so they are in the same GCS
sr = arcpy.SpatialReference(4269)

arcpy.Project_management(streams, workspace + "streams_project.shp", sr)


b_copy = arcpy.CopyFeatures_management(building_footprints, workspace + "buildings_copy")


arcpy.Project_management(b_copy, workspace+"buildings_project.shp", sr)



In [ ]:
#use near tool to get distance to steams
arcpy.Near_analysis(workspace+"buildings_project.shp",
                    workspace + "streams_project.shp",
                    method="GEODESIC")



In [ ]:
subwatershed_list = [] 
dist_to_stream_avg_list = []
dist_to_stream_stdev_list =[]


In [ ]:
#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)
        
        arcpy.ClipFeatures_management(workspace+"buildings_project.shp", row, f"{subwatershed_number}_buildings.shp")

        #get all the distances in to the streams
        dist_to_stream_list = []
        with arcpy.da.SearchCursor(workspace+f"{subwatershed_number}_buildings.shp", ["NEAR_DIST"]) as distCursor: 
            for dist in distCursor:  
                dist_to_stream = dist[0]  
                dist_to_stream_list.append(dist_to_stream)  

        dist_to_stream_avg = mean(dist_to_stream_list)

        dist_to_stream_stdev = std(dist_to_stream_list)

        # print(dist_to_stream_avg)
        # print(dist_to_stream_stdev)

        dist_to_stream_avg_list.append(dist_to_stream_avg)
        dist_to_stream_stdev_list.append(dist_to_stream_stdev)